<a href="https://colab.research.google.com/github/hsandaver/essays/blob/main/DyeClassifierWITHSmilesv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Step 1: Chromophore Sensitivity Dictionary

chromophore_sensitivity = {
    "azo": {"light_sensitivity": "high", "humidity_sensitivity": "medium"},
    "anthraquinone": {"light_sensitivity": "medium", "humidity_sensitivity": "high"},
    "methine": {"light_sensitivity": "high", "humidity_sensitivity": "medium"},
    "triarylmethane": {"light_sensitivity": "medium", "humidity_sensitivity": "high"},
    "stilbene": {"light_sensitivity": "high", "humidity_sensitivity": "medium"},
    "phthalocyanine": {"light_sensitivity": "low", "humidity_sensitivity": "low"}
}

# Default values for unknown chromophores
default_sensitivity = {"light_sensitivity": "unknown", "humidity_sensitivity": "unknown"}

# Step 2: Delta-E Calculation (CIE76)
def calculate_delta_e(input_lab, dataset_lab):
    """
    Calculate Delta-E (CIE76) color difference between input LAB and dataset LAB.

    Parameters:
        input_lab (list): The input LAB color from the colorimeter.
        dataset_lab (list): A list of LAB values from the dye dataset.

    Returns:
        float: Delta-E value.
    """
    delta_e = np.sqrt(
        (dataset_lab[0] - input_lab[0]) ** 2 +
        (dataset_lab[1] - input_lab[1]) ** 2 +
        (dataset_lab[2] - input_lab[2]) ** 2
    )
    return delta_e

# Step 3: Find closest dyes with sensitivity information

def find_closest_dyes_with_sensitivity(input_lab, dye_color_lab_map, dye_info_df, top_n=3):
    """
    Find the closest dyes based on Delta-E, and check for chromophore sensitivity.

    Parameters:
        input_lab (list): The input LAB color from the colorimeter.
        dye_color_lab_map (dict): The dye color LAB map with LAB values.
        dye_info_df (pd.DataFrame): The dye dataset with chromophore information.
        top_n (int): Number of top matches to return.

    Returns:
        list: Closest dye matches with sensitivity analysis.
    """
    matches = []

    for dye_name, dye_lab in dye_color_lab_map.items():
        delta_e = calculate_delta_e(input_lab, dye_lab)
        matches.append((dye_name, delta_e))

    matches = sorted(matches, key=lambda x: x[1])
    top_matches = matches[:top_n]

    # Add chromophore sensitivity analysis
    results_with_sensitivity = []
    for dye_name, delta_e in top_matches:
        chromophore = dye_info_df.loc[dye_info_df['Dye Name'] == dye_name, 'Chromophore'].values[0]

        # Look up sensitivity information based on chromophore
        sensitivity = chromophore_sensitivity.get(chromophore.lower(), default_sensitivity)
        results_with_sensitivity.append({
            "dye_name": dye_name,
            "delta_e": delta_e,
            "chromophore": chromophore,
            "light_sensitivity": sensitivity["light_sensitivity"],
            "humidity_sensitivity": sensitivity["humidity_sensitivity"]
        })

    return results_with_sensitivity

# Step 4: Display Sensitivity Warnings

def display_sensitivity_warnings(dye_matches):
    """
    Display warnings based on dye chromophore sensitivity to light and humidity.

    Parameters:
        dye_matches (list): List of dye matches with sensitivity information.
    """
    for dye in dye_matches:
        print(f"Dye: {dye['dye_name']}, Delta-E: {dye['delta_e']:.2f}")
        print(f"   Chromophore: {dye['chromophore']}")
        print(f"   Light Sensitivity: {dye['light_sensitivity']}")
        print(f"   Humidity Sensitivity: {dye['humidity_sensitivity']}")

        if dye["light_sensitivity"] == "high":
            print("   ⚠️ Warning: This dye is highly sensitive to light. Limit UV exposure.")

        if dye["humidity_sensitivity"] == "high":
            print("   ⚠️ Warning: This dye is highly sensitive to humidity. Control moisture levels.")

        print("")  # Empty line for better formatting

# Step 5: Visualize LAB Color Analysis (as per your previous LAB analyzer)

def create_lab_comparison_plot(input_rgb, closest_rgb, input_lab, closest_lab, closest_color_name, delta_e):
    """
    Creates a comparison plot displaying input and closest colors with annotations.
    """
    data = pd.DataFrame({
        'Color Type': ['Input Color', f'Closest: {closest_color_name}'],
        'RGB': [f'rgb{input_rgb}', f'rgb{closest_rgb}'],
        'LAB': [f'L={input_lab[0]}, A={input_lab[1]}, B={input_lab[2]}',
                f'L={closest_lab[0]}, A={closest_lab[1]}, B={closest_lab[2]}'],
        'Delta-E': [delta_e, 'N/A']
    })

    fig = px.scatter(
        data_frame=data,
        x=[0, 1],
        y=[1, 1],
        color='RGB',
        hover_data=['Color Type', 'LAB', 'Delta-E'],
        labels={'x': '', 'y': ''},
        title='🖌️ Input Color vs Closest Color',
    )

    fig.update_traces(marker=dict(size=50, line=dict(width=2, color='DarkSlateGrey')))
    fig.update_layout(
        showlegend=False,
        xaxis=dict(showticklabels=False, showgrid=False, zeroline=False),
        yaxis=dict(showticklabels=False, showgrid=False, zeroline=False),
        template='plotly_dark',
        margin=dict(l=50, r=50, t=80, b=50)
    )

    fig.add_annotation(
        x=0,
        y=1.05,
        text='Input Color',
        showarrow=False,
        font=dict(size=14, color='white'),
        xanchor='center'
    )
    fig.add_annotation(
        x=1,
        y=1.05,
        text=f'Closest: {closest_color_name}',
        showarrow=False,
        font=dict(size=14, color='white'),
        xanchor='center'
    )

    return fig

# Step 6: Example Workflow Integration

# Example Dye Color LAB Map (replace with real data)
dye_color_lab_map = {
    "Dye A": [26.85, 22.22, 6.88],
    "Dye B": [47.19, 38.18, 18.09],
    "Dye C": [27.65, 29.36, 8.72]
}

# Example DataFrame with chromophore information
dye_info_df = pd.DataFrame({
    "Dye Name": ["Dye A", "Dye B", "Dye C"],
    "Chromophore": ["azo", "anthraquinone", "phthalocyanine"]
})

# Input LAB color from colorimeter
input_lab = [50, -10, 30]

# Find the closest dyes and their sensitivities
dye_matches = find_closest_dyes_with_sensitivity(input_lab, dye_color_lab_map, dye_info_df)

# Display warnings based on sensitivity
display_sensitivity_warnings(dye_matches)

# Assuming the first match is the closest
closest_match = dye_matches[0]
input_rgb = [120, 100, 150]  # Example RGB for input (replace with conversion)
closest_rgb = [200, 50, 100]  # Example RGB for closest match (replace with conversion)
input_lab = [50, -10, 30]  # Input LAB
closest_lab = dye_color_lab_map[closest_match['dye_name']]

# Create comparison plot
fig = create_lab_comparison_plot(input_rgb, closest_rgb, input_lab, closest_lab, closest_match['dye_name'], closest_match['delta_e'])
fig.show()